In [ ]:
# https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
mnist.load_data()

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train.shape
y_train.shape

x_test.shape
y_test.shape

In [ ]:
x_train[0]

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

def gen_image(arr):
    two_d = (np.reshape(arr, (28, 28)))
    plt.imshow(two_d, cmap='gray')
    plt.axis('off')
    return plt

In [ ]:
for a in x_train[np.random.randint(0, x_train.shape[0], 5)]:
    gen_image(a).show()

In [ ]:
fig = plt.figure(figsize=(16, 8))

columns = 10
rows = 5

nums = x_train[np.random.randint(0, x_train.shape[0], columns * rows)]

for i in range(0, columns*rows):
    f = fig.add_subplot(rows, columns, i+1)
    d = gen_image(nums[i])
plt.show()

In [ ]:
def get_batch(x_data, y_data, batch_size):
    idxs = np.random.randint(0, len(y_data), batch_size)
    return x_data[idxs,:,:], y_data[idxs]

In [ ]:
learning_rate = 0.5
epochs = 10
batch_size = 100

x = tf.placeholder(tf.float32, [None, 28, 28])

x_rs = tf.reshape(x, (-1 , 784))
x_rs

# Scale down to [0,1]
x_sc = tf.div(x_rs, 255)

y = tf.placeholder(tf.int64, [None, 1])

y_one_hot = tf.reshape(tf.one_hot(y, 10), [-1, 10])


In [ ]:
W1 = tf.Variable(tf.random_normal([784, 300], stddev = 0.03), name = "W1")
b1 = tf.Variable(tf.random_normal([300], stddev = 0.03), name = "b1")

hidden = tf.add(tf.matmul(x_sc, W1), b1)
hidden = tf.nn.relu(hidden)

W2 = tf.Variable(tf.random_normal([300, 10], stddev = 0.03), name = "W2")
b2 = tf.Variable(tf.random_normal([10], stddev = 0.03), name = "b2")

logits = tf.add(tf.matmul(hidden, W2), b2)

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_one_hot, logits = logits))

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

In [ ]:
init_op = tf.global_variables_initializer()

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_one_hot, 1), tf.argmax(logits, 1))

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
import datetime

In [ ]:
total_batch = int(len(y_train) / batch_size)

print("Batch: {}, Total batch: {}\n".format(batch_size, total_batch))

print(datetime.datetime.now())

In [ ]:
with tf.Session() as session:
    session.run(init_op)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = get_batch(x_train, y_train, batch_size = batch_size)
            _, c = session.run([optimizer, cross_entropy], feed_dict={x: batch_x, y: batch_y.reshape(-1,1)})
            avg_cost += c / total_batch
        acc = session.run(accuracy, feed_dict = {x: x_test, y: y_test.reshape(-1,1)})
        print("Epoch: {}, cost : {:.3f}, accuracy: {:.3f}%".format(epoch + 1, avg_cost, acc * 100))

In [ ]:
print("\nTraining complete\n")

print(datetime.datetime.now())